# Hybrid/Solid Motor
Written by Brendan

## Imported Modules

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ipywidgets as widgets
import time
import math
from IPython.display import display

## Atmo Tables

In [2]:
SIAltTable=[i*1000 for i in [0,1,3,5,10,25,50,75,100,130,160,200,300,400,600,1000]]
SITempTable=[288.15,281.651,268.65,255.65,223.252,221.552,270.65,206.65,195.08,469.27,696.29,845.56,976.01,995.83,999.85,1000]
SIPressTable=[i*101325 for i in [1,0.887,0.66919,0.53313,0.26151,0.025158,0.00078735,0.000020408,0.00000031593,1.2341e-8,2.9997e-9,8.3628e-10,8.6557e-11,1.4328e-11,8.1056e-13,7.4155e-14]]
USAltTable=[i*3.28084 for i in SIAltTable]
USTempTable=[i*1.8 for i in SITempTable]
USPressTable=[i*0.020885434273039 for i in SIPressTable]

AltTable=SIAltTable
TempTable=SITempTable
PressTable=SIPressTable
    
%run RocketEquations.py


## Code

In [3]:
Metric = widgets.Button(description="Run Data")
outputMetric = widgets.Output(layout={'border': '1px solid black'})

def SI(b):
    with outputMetric:
        outputMetric.clear_output()
# Table Selection
        
        AltTable=SIAltTable
        TempTable=SITempTable
        PressTable=SIPressTable
        
# Fuel Selection
        Prop=PropType(Oxidizer.value,Fuel.value)
        r=Prop[0]      # Mass prop ratio
        rV=Prop[1]     # Volume prop ratio
        SG=Prop[2]     # Average specific gravity
        t1=Prop[3]     # Combustion Temperature K
        #cst=Prop[4]    # C Star (speed of sound / thrust coeff)
        MW=Prop[5]     # Molar Mass kg/mol
        Isp=Prop[6]    #Specific Impulse
        k=Prop[7]      # Ratio of Specific Heats
        
# Pre-setting Data Point Arrays
        X=[]
        Y=[]
        X15=[]
        Y15=[]
        
# Constants
        g=9.81         # m/s^2
        R=8314         # kJ/mol-K
        VCF=1       # Velocity Correction
        TCF=1       # Thrust Correction
        
# Calculating Required Parameters and Tabulating
        F_IDEAL=Thrust.value  # N
        p1=689475.7*ChamberPressure.value # Pa
        p2=101325*Ambient.value       # Pa
        p3=p2
        ToF=60     # seconds
        
        denO=1141
        denF=810        
        dwo=denO*g         # Weight Density 
        dwf=denF*g         # Weight Density 
        
        F_TRUE=F_IDEAL*TCF
        p2p1=p2/p1
        
        v2IDEAL=ExhaustVMol(k,R,t1,MW,p2p1)
        v2TRUE=v2IDEAL*VCF
        
        IspIDEAL=SpecificImpulseV(v2IDEAL,g)
        IspTRUE=SpecificImpulseV(v2TRUE,g)
        
        CfIDEAL=CfIdeal(k,p2p1)
        CfTRUE=CfIDEAL*TCF
        
        AtIDEAL=F_IDEAL/(CfIDEAL*p1)
        AtTRUE=F_IDEAL/(CfTRUE*p1)
        AR=AreaRatio(k,p2p1)
        if AR>ARLimit.value:
            AR=ARLimit.value
        A2IDEAL=AtIDEAL*AR
        A2TRUE=AtTRUE*AR
        
        M2=1
        Res=0.0001
        ARint=AR**-1
        
        while ARint<AR:
            M2=M2+Res
            ARint=ExpansionRatio(k,M2)
            
        mdotIDEAL=F_IDEAL/v2IDEAL
        mdotTRUE=F_TRUE/v2TRUE
        
        wdtIDEAL=mdotTRUE*g                  # Total Weight Flow Rate
        wdtTRUE=mdotIDEAL*g  
        wdoIDEAL=OxWdot(wdtIDEAL,r)          # Ox Weight Flow Rate
        wdoTRUE=OxWdot(wdtTRUE,r)
        wdfIDEAL=FuelWdot(wdtIDEAL,r)        # Fuel Weight Flow Rate
        wdfTRUE=FuelWdot(wdtTRUE,r)
        
        wpIDEAL=wdtIDEAL*ToF                 # Prop Weight
        wpTRUE=wdtTRUE*ToF
        woIDEAL=wdoIDEAL*ToF                 # Ox Weight
        woTRUE=wdoTRUE*ToF
        wfIDEAL=wdfIDEAL*ToF                 # Fuel Weight
        wfTRUE=wdfTRUE*ToF
        
        VdoIDEAL=wdoIDEAL/dwo                # Ox Volume Flow Rate
        VdoTRUE=wdoTRUE/dwo
        VdfIDEAL=wdfIDEAL/dwf                # Fuel Volume Flow Rate
        VdfTRUE=wdfTRUE/dwf
        
        VoIDEAL=VdoIDEAL*ToF                 # Ox Volume 
        VoTRUE=VdoTRUE*ToF
        VfIDEAL=VdfIDEAL*ToF                 # Fuel Volume
        VfTRUE=VdfTRUE*ToF
            

        
# Calculating Nozzle Dimensions
        Rt=(AtTRUE/3.141592)**0.5 # m
        Re=Rt*(AR)**0.5           # m
        PL15=L15.value
        L=PL15*((Re-Rt)/math.tan(math.radians(15))) # SCALING FACTOR
        Angle=LARA(PL15,AR)
        AngleN=Angle[0]                 # Degrees
        AngleE=Angle[1]
# Converging Nozzle
        for i in range (-135,-90,1):
            X.append(1.5*Rt*math.cos(math.radians(i)))
            Y.append(1.5*Rt*math.sin(math.radians(i))+1.5*Rt+Rt)
            X15.append(1.5*Rt*math.cos(math.radians(i)))
            Y15.append(1.5*Rt*math.sin(math.radians(i))+1.5*Rt+Rt)

# Throat and Circular Divergence
        for i in range (-90,(AngleN-90),1):
            X.append(0.382*Rt*math.cos(math.radians(i)))
            Y.append(0.382*Rt*math.sin(math.radians(i))+0.382*Rt+Rt)

        for i in range (-90,(15-90),1):    
            X15.append(0.382*Rt*math.cos(math.radians(i)))
            Y15.append(0.382*Rt*math.sin(math.radians(i))+0.382*Rt+Rt)

# Bell Divergence
        Ex=L
        Ey=Re
        Nx=0.382*Rt*math.cos(math.radians(AngleN-90))
        Ny=0.382*Rt*math.sin(math.radians(AngleN-90))+0.382*Rt+Rt
        m1=math.tan(math.radians(AngleN))
        m2=math.tan(math.radians(AngleE))
        C1=Ny-m1*Nx
        C2=Ey-m2*Ex
        Qx=(C2-C1)/(m1-m2)
        Qy=(m1*C2-m2*C1)/(m1-m2)

        for t in range (0,1000,1):
            t=t/1000
            X.append(Qx+(1-t)**2*(Nx-Qx)+t**2*(Ex-Qx))
            Y.append(Qy+(1-t)**2*(Ny-Qy)+t**2*(Ey-Qy))
            
# Combustion Chamber Work
        Dt=Rt*2
        Dc=Dt*2
        Rc=Dt
        Ac=0.5*math.pi*Rc**2
        Beta=45
        
        Lst=Lstar.value #m
        Vchamber=Lst*AtTRUE
        Lconv=-Rc/math.tan(math.radians(Beta))
        Vconv=(1/3)*math.pi*(Rc**2+Rc*Rt+Rt**2)*Lconv
        
        Vcyl=Vchamber-Vconv
        Lcyl=Vcyl/Ac
        
        Lchamber=Lcyl+Lconv
        
        CDXinit=-Lchamber
        CDX=[CDXinit]
        CDY=[Rc]
        CDXcrv=X[0]-(Rc-Y[0])
        
        while CDXinit<CDXcrv:
            CDXinit=CDXinit+0.00001
            CDY.append(Dt)
            CDX.append(CDXinit)
            
        CDYinit=Rc
        
        while CDXinit<X[0]:
            CDXinit=CDXinit+0.00001
            CDYinit=CDYinit-0.00001
            CDY.append(CDYinit)
            CDX.append(CDXinit)
        
# Plotting the Bell Nozzle 
        NegY=[i*-1 for i in Y]
        NegCDY=[i*-1 for i in CDY]
        plt.xlabel('Length in Meters')
        plt.ylabel('Width in Meters')
        plt.plot(X,Y,color='tab:gray')
        plt.plot(CDX,CDY,color='tab:gray')
        plt.plot(X,NegY,color='tab:gray')
        plt.plot(CDX,NegCDY,color='tab:gray')
        plt.xlim([-3.76631*Re*.75,7.0433*Re*.75])
        plt.ylim([-3.48458*Re*.75,3.48458*Re*.75])
        plt.title('Rao Nozzle Plot')
        plt.show()

# 15 Degree Half Angle Divergence
        Nx15=0.382*Rt*math.cos(math.radians(15-90))
        Ny15=0.382*Rt*math.sin(math.radians(15-90))+0.382*Rt+Rt

        xtemp=Nx15
        while math.tan(math.radians(15))*xtemp+Ny15 < Re:
            xtemp=xtemp+0.00001
            Y15.append(math.tan(math.radians(15))*xtemp+Ny15-math.tan(math.radians(15))*Nx15)
            X15.append(xtemp)
            

# Plotting 15 degree Half Angle Nozzle
        NegY15=[i*-1 for i in Y15]
        plt.xlabel('Length in Meters')
        plt.ylabel('Width in Meters')
        plt.plot(X15,Y15,color='tab:gray')
        plt.plot(CDX,CDY,color='tab:gray')
        plt.plot(X15,NegY15,color='tab:gray')
        plt.plot(CDX,NegCDY,color='tab:gray')
        plt.xlim([-3.76631*Re*.75,7.0433*Re*.75])
        plt.ylim([-3.48458*Re*.75,3.48458*Re*.75])
        plt.title('15 Degree Angle Nozzle Plot')
        plt.show()
        
# Combustion Instability
        RadialL=2*(Dc)
        TanL=2*math.pi*(Dc)
        AxialL=2*(Lchamber)
        
        a=(k*t1*R/MW)**0.5
        
        RadialF=a/RadialL
        TanF=a/TanL
        AxialF=a/AxialL
        
# Turbo Pumps and Feed Systems
        # Ox Pump
        OxTankP=101325*3                  # (Pa)
        OxHeadS=OxTankP/1 # (m) METRIC ONLY
        
        
        # Fuel Pump
        
        # Transit Passages
        ToCool=(2*((VdfTRUE/5)*(1000*1000))/math.pi)**0.5 # Radius of passages leading in. Divide as needed.
        # Cooling Passages
        RegW=2
        RegH=2
        Aregen=RegW*RegH                       # Cooling Passage width mm^2
        flow10A=(VdfTRUE/10)*(1000*1000)       # Total Passage x-Area Needed at 10 m/s mm^2
        flow20A=(VdfTRUE/20)*(1000*1000)       # mm^2
        flow5A=(VdfTRUE/5)*(1000*1000)         # mm^2
        MinRad10A=(flow10A/Aregen)*(RegW+2*0.5)/(2*math.pi) # Minimum radius to implement mm
        MinRad20A=(flow20A/Aregen)*RegW/(2*math.pi) # mm
        MinRad5A=(flow5A/Aregen)*RegW/(2*math.pi)   # mm
        
        print('---------------------------------------------------------')
        print('---------Velocities, Areas, Flowrates, Weights-----------')
        print('---------------------------------------------------------\n')
        
        print('Value                      Ideal         Actual')
        print('Exit Velocity:             ','{0:.0f}'.format(v2IDEAL),'(m/s)    '
              '{0:.0f}'.format(v2TRUE),'(m/s)\n')
        print('Specific Impulse:             ','{0:.1f}'.format(IspIDEAL),'(m/s)    '
              '{0:.0f}'.format(IspTRUE),'(m/s)\n')
        print('Thrust Coeff:             ','{0:.2f}'.format(CfIDEAL),'(~)    '
              '{0:.2f}'.format(CfTRUE),'(m/s)\n')
        print('Throat Area:               ','{0:.8f}'.format(AtIDEAL),'(m^2)      '
              '{0:.8f}'.format(AtTRUE),'(m^2)\n')
        print('Exit Area:                 ','{0:.6f}'.format(A2IDEAL),'(m^2)    '
              '{0:.6f}'.format(A2TRUE),'(m^2)\n')
        print('Exit Mach:                 ','{0:.2f}'.format(M2),'(~)        '
              '{0:.2f}'.format(M2),'(~)\n')
        print('Total Weight Flow Rate:    ','{0:.2f}'.format(wdtIDEAL),'(N/s)    '
              '{0:.2f}'.format(wdtTRUE),'(N/s)\n')
        print('Oxidizer Weight Flow Rate: ','{0:.2f}'.format(wdoIDEAL),'(N/s)    '
              '{0:.2f}'.format(wdoTRUE),'(N/s)\n')
        print('Fuel Weight Flow Rate:     ','{0:.2f}'.format(wdfIDEAL),'(N/s)    '
              '{0:.2f}'.format(wdfTRUE),'(N/s)\n')
        print('Propellant Weight:         ','{0:.0f}'.format(wpIDEAL),'(N)    '
              '{0:.0f}'.format(wpTRUE),'(N)\n')
        print('Oxidizer Weight:           ','{0:.0f}'.format(woIDEAL),'(N)     '
              '{0:.0f}'.format(woTRUE),'(N)\n')
        print('Fuel Weight:               ','{0:.0f}'.format(wfIDEAL),'(N)     '
              '{0:.0f}'.format(wfTRUE),'(N)\n')
        print('Oxidizer Volume Flow Rate: ','{0:.6f}'.format(VdoIDEAL),'(m^3/s)    '
              '{0:.4f}'.format(VdoTRUE),'(m^3/s)\n')
        print('Fuel Volume Flow Rate:     ','{0:.6f}'.format(VdfIDEAL),'(m^3/s)    '
              '{0:.4f}'.format(VdfTRUE),'(m^3/s)\n')
        print('Oxidizer Volume:           ','{0:.4f}'.format(VoIDEAL),'(m^3)    '
              '{0:.4f}'.format(VoTRUE),'(m^3)\n')
        print('Fuel Volume:               ','{0:.4f}'.format(VfIDEAL),'(m^3)    '
              '{0:.4f}'.format(VfTRUE),'(m^3)\n')
        
        print('Expansion Ratio:',AR)
        
        print('Nozzle Exit Diameter:',Re*2)
        
        print('---------------------------------------------------------')
        print('-------------------------Frequencies---------------------')
        print('---------------------------------------------------------\n')
        print('Nozzle Length:','{0:.5f}'.format(L),'(m)\n')
        
        print('L* (Assumed):',Lst,'(m)')
        print('Convergence Angle (Assumed): 42 (deg)')
        print('Converging Duct Length (Calculated):','{0:.5f}'.format(-CDXcrv),'(m)')
        print('Cylindrical Duct Length (Calculated):',Lcyl,'(m)')
        print('Chamber Diameter (Calculated):','{0:.7f}'.format(Dc),'(m)\n')
        
        print('Radial Resonance Frequency:','{0:.2f}'.format(RadialF),'(Hz)')
        print('Tangential Resonance Frequency:','{0:.2f}'.format(TanF),'(Hz)')
        print('Axial Resonance Frequency:','{0:.2f}'.format(AxialF),'(Hz)')
        
        print('Mass Flow Rate',mdotTRUE)
        
        print('\n---------------------------------------------------------')
        print('-----------------Pumps and Passages----------------------')
        print('---------------------------------------------------------\n')
        
        print('Total Area for 5m/s:', flow5A,'(mm^2)\n')
        print('Minimum Radius for 5m/s:', MinRad5A,'(mm)\n\n')
        
        print('Total Area for 10m/s:', flow10A,'(mm^2)\n')
        print('Minimum Radius for 10m/s:', MinRad10A,'(mm)\n\n')
        
        print('Total Area for 20m/s:', flow20A,'(mm^2)\n')
        print('Minimum Radius for 20m/s:', MinRad20A,'(mm)\n\n\n')
        
        print('Pump to Regen Radius:', ToCool,'(mm)')

Metric.on_click(SI)

## GUI

In [4]:
Oxidizer=widgets.Dropdown(
    options=['N/A','Nitrous Oxide'],
    value='Nitrous Oxide',
    description='Oxidizer:',
    disabled=False,
)

Fuel=widgets.Dropdown(
    options=['Sorbitol','ABS'],
    value='ABS',
    description='Fuel:',
    disabled=False,
)

Thrust=widgets.FloatText(
    value=200,
    description='Thrust (N): ',
    disabled=False,
)

ChamberPressure=widgets.FloatText(
    value=6,
    description='Chamber 100 psi: ',
    disabled=False,
)

L15=widgets.FloatText(
    value=0.85,
    description='% of 15 deg: ',
    disabled=False,
)

Ambient=widgets.FloatText(
    value=0.95,
    description='Ambient: ',
    disabled=False,
)

Lstar=widgets.FloatText(
    value=0.05,
    description='L*: ',
    disabled=False,
)

ARLimit=widgets.FloatText(
    value=100,
    description='Max AR: ',
    disabled=False,
)

DispWindows1=widgets.HBox([Oxidizer,Fuel],
                        layout=widgets.Layout(overflow='visible',
                        align_items='center',
                        align_contents='center',)
)

DispWindows2=widgets.HBox([Thrust,ChamberPressure],
                        layout=widgets.Layout(overflow='visible',
                        align_items='center',
                        align_contents='center',)
)

DispWindows3=widgets.HBox([L15,Ambient],
                        layout=widgets.Layout(overflow='visible',
                        align_items='center',
                        align_contents='center',)
)

DispWindows4=widgets.HBox([ARLimit,Lstar],
                        layout=widgets.Layout(overflow='visible',
                        align_items='center',
                        align_contents='center',)
)
display(DispWindows1)
display(DispWindows2)
display(DispWindows3)
display(DispWindows4)
display(Metric, outputMetric)




Button(description='Run Data', style=ButtonStyle())

Output(layout=Layout(border='1px solid black'))